In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Alopecia"
cohort = "GSE148346"

# Input paths
in_trait_dir = "../../input/GEO/Alopecia"
in_cohort_dir = "../../input/GEO/Alopecia/GSE148346"

# Output paths
out_data_file = "../../output/preprocess/Alopecia/GSE148346.csv"
out_gene_data_file = "../../output/preprocess/Alopecia/gene_data/GSE148346.csv"
out_clinical_data_file = "../../output/preprocess/Alopecia/clinical_data/GSE148346.csv"
json_path = "../../output/preprocess/Alopecia/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"JAK3/TEC and TYK2/JAK1 inhibitors demonstrate significant improvement in scalp alopecia areata biomarkers"
!Series_summary	"We present the biopsy sub-study results from the first randomized, placebo-controlled clinical trial in patients with alopecia areata (AA) with ≥50% scalp hair loss and ≤7 years since the last AA episode. In this sub-study, we evaluated the molecular responses to PF-06651600, an oral inhibitor of JAK3 and the tyrosine kinase expressed in hepatocellular carcinoma (TEC) kinase family, and PF-06700841, an oral TYK2/JAK1 inhibitor, versus placebo in nonlesional and lesional scalp biopsies of biopsy samples from patients with AA."
!Series_overall_design	"This is a novel design, phase 2a, multicenter study that evaluates the efficacy, safety, and tolerability of PF-06651600 and PF-06700841 versus placebo in patients with AA. The biopsy sub-study took place during the randomized, double-blind initial 24 weeks of the trial. 46 patien

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Check if this dataset likely contains gene expression data
is_gene_available = True  # Based on the Series_summary, this appears to contain gene expression data

# Analyzing trait, age, and gender data availability
# Looking at the Sample Characteristics Dictionary for relevant data rows

# For trait (Alopecia):
# Row 3 contains 'tissue disease state' which indicates lesional (LS) vs non-lesional (NL) 
# This can represent the trait status (LS = affected by Alopecia, NL = unaffected)
trait_row = 3

# Age data is not available in the sample characteristics
age_row = None

# Gender data is not available in the sample characteristics
gender_row = None

# Define conversion functions for available data
def convert_trait(value):
    """Convert trait values to binary format (0 for NL, 1 for LS)"""
    if not value or ":" not in value:
        return None
    
    # Extract value after colon
    value = value.split(":", 1)[1].strip()
    
    # Convert to binary
    if value == "NL":  # Non-lesional (unaffected)
        return 0
    elif value == "LS":  # Lesional (affected)
        return 1
    else:
        return None

def convert_age(value):
    """Convert age values to numeric format"""
    # Function placeholder since age data is not available
    return None

def convert_gender(value):
    """Convert gender values to binary format (0 for female, 1 for male)"""
    # Function placeholder since gender data is not available
    return None

# Determine trait data availability
is_trait_available = trait_row is not None

# Save metadata using the validate_and_save_cohort_info function
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Extract clinical features if trait data is available
if trait_row is not None:
    # Convert the sample characteristics dictionary to a DataFrame
    sample_char_dict = {0: ['patient_id: 10051003', 'patient_id: 10051004', 'patient_id: 10051005', 'patient_id: 10051006', 'patient_id: 10051007', 'patient_id: 10051008', 'patient_id: 10051009', 'patient_id: 10051010', 'patient_id: 10051012', 'patient_id: 10071001', 'patient_id: 10071002', 'patient_id: 10071003', 'patient_id: 10071007', 'patient_id: 10071009', 'patient_id: 10071010', 'patient_id: 10071011', 'patient_id: 10071013', 'patient_id: 10071014', 'patient_id: 10071015', 'patient_id: 10071016', 'patient_id: 10071017', 'patient_id: 10071018', 'patient_id: 10071019', 'patient_id: 10071020', 'patient_id: 10071022', 'patient_id: 10071023', 'patient_id: 10071024', 'patient_id: 10071025', 'patient_id: 10071026', 'patient_id: 10131003'], 1: ['batch_date: 2018-03-12', 'batch_date: 2018-03-13', 'batch_date: 2018-03-15', 'batch_date: 2018-03-26', 'batch_date: 2018-03-20', 'batch_date: 2018-03-22', 'batch_date: 2018-03-28'], 2: ['tissue: Skin biopsy'], 3: ['tissue disease state: LS', 'tissue disease state: NL'], 4: ['week: W0', 'week: W12', 'week: W24'], 5: ['treatment: PF06700841', 'treatment: PF06651600', 'treatment: Placebo']}
    
    # Create a DataFrame with the sample characteristics
    columns = []
    for i in range(max(sample_char_dict.keys()) + 1):
        if i in sample_char_dict:
            columns.append(sample_char_dict[i])
        else:
            columns.append([])
    
    clinical_df = pd.DataFrame(columns)
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_df,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the selected clinical data
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical data:")
    print(preview)
    
    # Save the selected clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical data:
{0: [1.0], 1: [0.0], 2: [nan], 3: [nan], 4: [nan], 5: [nan], 6: [nan], 7: [nan], 8: [nan], 9: [nan], 10: [nan], 11: [nan], 12: [nan], 13: [nan], 14: [nan], 15: [nan], 16: [nan], 17: [nan], 18: [nan], 19: [nan], 20: [nan], 21: [nan], 22: [nan], 23: [nan], 24: [nan], 25: [nan], 26: [nan], 27: [nan], 28: [nan], 29: [nan]}
Clinical data saved to ../../output/preprocess/Alopecia/clinical_data/GSE148346.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1294_at', '1316_at',
       '1320_at', '1405_i_at', '1431_at', '1487_at', '1552256_a_at',
       '1552257_a_at', '1552263_at', '1552264_a_at', '1552274_at',
       '1552275_s_at', '1552277_a_at', '1552280_at', '1552283_s_at',
       '1552286_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the first 20 gene/probe identifiers like '1007_s_at', '1053_at', etc.
# These appear to be Affymetrix probe set IDs (indicated by the '_at' suffix pattern),
# not human gene symbols. Affymetrix IDs need to be mapped to gene symbols
# for proper biological interpretation.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns with gene identifiers and gene symbols
# From the gene annotation preview, 'ID' contains probe IDs (like '1007_s_at') which match the gene identifiers in gene_data
# 'Gene Symbol' contains the gene symbols we need to map to
probe_id_col = 'ID'
gene_symbol_col = 'Gene Symbol'

# 2. Extract gene mapping dataframe with these two columns
gene_mapping = get_gene_mapping(gene_annotation, probe_id_col, gene_symbol_col)

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print the number of genes after mapping for verification
print(f"Number of genes after mapping: {len(gene_data)}")
print("First 10 gene symbols after mapping:")
print(gene_data.index[:10])


Number of genes after mapping: 15128
First 10 gene symbols after mapping:
Index(['A1BG-AS1', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1', 'A4GALT', 'AAAS',
       'AACS', 'AACSP1', 'AADAC'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("Normalizing gene symbols...")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to a CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
print("Loading the original clinical data...")
# Get the matrix file again to ensure we have the proper data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

print("Extracting clinical features...")
# Use the clinical_data obtained directly from the matrix file
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# Save the clinical data to a CSV file
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Link clinical and genetic data using the normalized gene data
print("Linking clinical and genetic data...")
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 3. Handle missing values in the linked data
print("Handling missing values...")
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 4. Check if trait is biased
print("Checking for bias in trait distribution...")
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Final validation
note = "Dataset contains gene expression data from bronchial brushings from control individuals and patients with asthma after rhinovirus infection in vivo, as described in the study 'Rhinovirus-induced epithelial RIG-I inflammasome suppresses antiviral immunity and promotes inflammation in asthma and COVID-19'."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

print(f"Dataset usability: {is_usable}")

# 6. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for trait-gene association studies due to bias or other issues.")

Normalizing gene symbols...
Gene data shape after normalization: (14601, 129)


Normalized gene data saved to ../../output/preprocess/Alopecia/gene_data/GSE148346.csv
Loading the original clinical data...
Extracting clinical features...
Clinical data preview:
{'GSM4462080': [1.0], 'GSM4462081': [1.0], 'GSM4462082': [1.0], 'GSM4462083': [1.0], 'GSM4462084': [1.0], 'GSM4462085': [1.0], 'GSM4462086': [1.0], 'GSM4462087': [0.0], 'GSM4462088': [1.0], 'GSM4462089': [1.0], 'GSM4462090': [1.0], 'GSM4462091': [1.0], 'GSM4462092': [1.0], 'GSM4462093': [1.0], 'GSM4462094': [1.0], 'GSM4462095': [1.0], 'GSM4462096': [1.0], 'GSM4462097': [0.0], 'GSM4462098': [1.0], 'GSM4462099': [1.0], 'GSM4462100': [1.0], 'GSM4462101': [1.0], 'GSM4462102': [1.0], 'GSM4462103': [1.0], 'GSM4462104': [0.0], 'GSM4462105': [1.0], 'GSM4462106': [1.0], 'GSM4462107': [1.0], 'GSM4462108': [1.0], 'GSM4462109': [1.0], 'GSM4462110': [1.0], 'GSM4462111': [1.0], 'GSM4462112': [1.0], 'GSM4462113': [1.0], 'GSM4462114': [1.0], 'GSM4462115': [1.0], 'GSM4462116': [1.0], 'GSM4462117': [1.0], 'GSM4462118': [1.0], 

Linked data shape after handling missing values: (129, 14602)
Checking for bias in trait distribution...
For the feature 'Alopecia', the least common label is '0.0' with 17 occurrences. This represents 13.18% of the dataset.
The distribution of the feature 'Alopecia' in this dataset is fine.

A new JSON file was created at: ../../output/preprocess/Alopecia/cohort_info.json
Dataset usability: True


Linked data saved to ../../output/preprocess/Alopecia/GSE148346.csv
